In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
sns.set(rc={"figure.figsize": (10, 6)})
sns.set_style("whitegrid")
sns.set_context("notebook")

In [4]:
posts = pd.read_hdf("metadata.h5", "posts")
tag_count = pd.read_hdf("metadata.h5", "tag_count")

In [5]:
useless_tags = [
    "highres",
    "bad_id",
    "bad_pixiv_id",
    "monochrome",
    "censored",
    "alternate_costume",
    "hetero",
    "sketch",
    "yuri",
    "character_name",
    "greyscale",
    "artist_name",
    "artist_request",
    "artist_request",
    "copyright_request",
    "absurdres",
    "dated",
    "signature",
    "cosplay",
    "translated",
    "copyright_name",
    "traditional_media",
    "twitter_username",
    "alternate_hairstyle",
    "mosaic_censoring",
    "parody",
    "english",
    "gradient",
    "couple",
    "sisters",
    "cover_page",
    "crossover",
    "uncensored",
    "official_art",
    "letterboxed",
    "translation_request",
    "scan",
    "game_cg",
    "remodel_(kantai_collection)",
    "depth_of_field",
    "convenient_censoring",
    "foreshortening",
    "watermark",
    "genderswap",
    "adapted_costume",
    "pov",
    "wallpaper",
    "pokemon_(creature)",
    "text",
    "kemonomimi_mode",
    "shinkaisei-kan",
    "genderswap_(mtf)",
    "personification",
    "blurry",
    "wind",
    "younger",
    # Tags for the background
    "simple_background",
    "white_background",
    "grey_background",
    "gradient_background",
    "blue_background",
    "pink_background",
    # And this is about the "camera" position
    "cowboy shot",
    "dutch_angle",
    "cover",
    "from_above",
    "from_below",
    "full_body",
    "upper_body"
]

In [6]:
tag_count_filtered = tag_count[tag_count.name.isin(useless_tags) == False].iloc[:512,:2]
tag_count_filtered.index = np.arange(1, len(tag_count_filtered) + 1)

In [7]:
print(tag_count_filtered.to_string())

                              name    count
1                            1girl  1424631
2                             solo  1224696
3                        long_hair   905280
4                          breasts   602468
5                            blush   572566
6                            smile   560367
7                       short_hair   547936
8                looking_at_viewer   449143
9                       open_mouth   433646
10                       blue_eyes   419919
11                     blonde_hair   412301
12                      brown_hair   381548
13                  multiple_girls   373391
14                           skirt   360861
15                      thighhighs   331289
16                        red_eyes   321309
17                             hat   316313
18                      black_hair   316213
19                   large_breasts   298863
20                          ribbon   264821
21                          2girls   255296
22                          glov